In [5]:
from random import randint
from playwright.async_api import async_playwright

In [6]:
SONG_URLS = [f'https://maistocadas.mus.br/{year}' for year in range(1958, 2024)]
SONGS_DIR = 'songs/'

def get_random_geolocation():
    return {'latitude': randint(-90,90), 'longitude':randint(-180,180)}

def get_random_viewport():
    return {'width': randint(800, 1920), 'height': randint(600, 1080)}

In [8]:
async with async_playwright() as playwright:
    browser = browser = await playwright.chromium.launch(headless = False)
    for url in SONG_URLS:
        page = await browser.new_page(viewport=get_random_viewport(), geolocation=get_random_geolocation())
        await page.goto(url)
        track_list = await page.locator('#tracklist').locator('li').all()
        title_and_artist_list = [f'"{str(await track_card.locator('span.musicas').text_content()).strip()}","{str(await track_card.locator('span.artista').text_content()).strip()}"\n'
                                  for track_card in track_list]
        await page.close()
        with open(SONGS_DIR+url.split('/')[-1]+'.csv', 'w') as f:
            f.write('title,artist\n')
            f.writelines(title_and_artist_list)